### Slew Frame Stacking and Astrometry.Net Call 

#### Loose methodology to stack slew frames together and get a refined aspect solution: 
1. identify slew frames by comparing time stamps of aspect parquet and scst raw aspect solution (frames in scst
    file and not aspect parquet are considered slew frames if they have the correct voltage) 
2. make 2s backplanes and .1 s backplanes made for the correct leg / eclipse / frames ID'd above 
3. filter 2s backplane image to make it just blurry enough for ASTRIDE to work 
4. run ASTRIDE streak detection on the 2s backplanes to get streaks from image (ASTRIDE uses edge detection) 
5. use streaks from ASTRIDE to get direction of movement (aka a linear best fit for streaks) 
6. stack / combine 10 or 20 (still deciding) .1s frames to make a more "pointlike" image 
7. use DAOstarfinder to get coordinates from stacked image and run Astrometry.net on that xylist OR run Astrometry.net on the stacked image 
8. backtrack to figure out which pixel in the stacked image is the "center" of the image and which timestamp
    it correlates with 
9. edit aspect parquet with the astrometry.net solution 
10. run gphoton with edited aspect parquet 
11. run gphoton with OG aspect soln from SCST file 
12. check and compare FWHM of outputs from 10 and 11 

### TO DO: 
- edit backplanes code to allow subsections of eclipses to have backplanes made for them instead of the whole thing
  (saves time and memory) 
  
- automate how the SCST file is downloaded 

In [ ]:
"""
Pipeline Settings
eclipse: just the int
leg: not required, but only if you want to ONLY do a specific leg of an eclipse 
astrometry:if you want an XY list or image run with astrometry.net 
compare: if you want a quality check on the outputs by running gphoton and comparing with OG SCST file soln 
"""
eclipse = 
leg = 
astrometry = 
compare = 

In [ ]:
# Eclipse info 
""" Get eclipse info: how many legs, length, flags, etc """


In [ ]:
# SCST download 
""" Download SCST file from MAST """

# scst files 
paths = get_raw_paths(eclipse)
paths['scst']

In [ ]:
# Some kind of loop to iterate the rest of the pipeline over each leg 
# add some kind of system to ignore leg-specific failures or report problems 

def meta_slew_pipeline(): 
    """for iterating over multiple chunks / legs of slew frames in an eclispe """
    # get eclipse info 
    info = get_eclipse_info()
    # get slew frames 
    slew_frames = get_slew_data(eclipse, band, scstpath)
    # loop through slew stacking pipeline for multiple legs 
    for i in range(legs):
        print(f"Running slew stacking pipeling for leg {i}.")
        slew_stacking_pipeline()
    print("Completed slew pipeline.")
    return 

In [ ]:
def slew_stacking_pipeline(): 
    """main pipeline for processing a chunk of slew frames from an eclipse. 
     will have to be run multiple times for eclipses with multiple legs (ie AIS etc)
     that have slew frames between legs. """
    
    # generate file names 
    
    # backplanes call 
    generate_backplanes() 
    
    
    
    

In [ ]:
def make_file_names(): 
    files = {'aspect1': , 
             'aspect2': , 
             'scst': , 
             'shortBackplanes': , 
             'longBackplanes': , 
             'smoothedLongBackplanes':
             'streaks': }

In [ ]:
def get_slew_data(eclipse, band, scstpath):
    """
    Use SCST file to get missing slew data aka timestamps not available in
    the refined aspect soln that are in the scst file and are at HVNOM. 
    """
    from gPhoton.io.mast import get_raw_paths, download_data 
    #download scst 
    scst = download_data(
        eclipse, "scst", band, datadir=os.path.dirname(scstpath),
    )
    scst_pd = pd.DataFrame(scst[1].data)
    scst_pd = scst_pd.rename(columns={"pktime": "time"})
    
    # loading aspect table 
    parq  = parquet.read_table('/home/bekah/gphoton_working/gPhoton/aspect/aspect.parquet')
    aspect = parq.to_pandas()
    aspect = aspect[aspect["eclipse"]==eclipse]
    aspect = aspect.reset_index()
    
    #merge to get slew frames 
    merged_df = scst_pd.merge(aspect, how = 'left', on = ['time'])
    slew_frames = slew_frames[slew_frames['hvnom_nuv']==1]

    return slew_frames

In [ ]:
# Actual backplanes call (probably currently a subprocess command but eventually it will all be one pipeline)

# to edit in backplanes file: 
# I think I can modify the t min and t max to specify for specific slew frame ranges 
# that would mean multiple separate calls to the backplanes module for a single eclipse
# but that doesn't seem sig less efficient than running on multiple ranges within one backplanes call
# the main difference would be loading the photonlist multiple times, I think 
# but ideally thiis is all only run once ... 

end_to_shared_memory(components, depth):
    total_trange = (components['t'].min(), components['t'].max()) 

In [ ]:
# Make two different kinds of backplanes and save them in a well-organized folder system 

def generate_backplanes(eclipse, leg, ): 
    """
    these are the current backplanes inputs: 
            make_backplanes(
                eclipse=eclipse,
                band=band,
                depth=1,
                leg=0,
                threads=4,
                burst=True,
                local="/home/bekah/gphoton_working/test_data",
                kind="dose",
                radius=400,
                write={'array': False, 'xylist': True},
                inline=True,
                threshold=.75,
                star_size=2
     """
    

In [ ]:
# Filter 2s backplane frame 

def filter_image(files):
    """use gaussian filter to smooth image for better processing"""
    from astropy.io import fits
    from skimage import filters

    dose_ais = fits.open(f"/home/bekah/gphoton_working/test_data/e10982/e10982-nd-t0002-b01-f00{f}-g_dose.fits.gz")

    smooth = filters.gaussian(dose_ais[0].data, sigma=2)

    hdu = fits.PrimaryHDU(smooth)
    hdul = fits.HDUList([hdu])
    hdul.writeto(f'{smoothed}.fits')
    
    return 

In [ ]:
# Run ASTRIDE on 2s backplane (requires a special conda env)

In [ ]:
def streak_and_stack(): 
    """looks at ASTRIDE results and uses streak length / direction to
    stack .1s frames into a 1s image """
    streaks = pd.read_csv(f"/home/bekah/glcat/smoothf{f}/streaks.csv") 
    # calculate offsets 
    streaks['x_diff']=streaks['x_min']-streaks['x_max']
    streaks['y_diff']=streaks['y_min']-streaks['y_max']
    # max offset 
    ymax = max(abs(streaks['y_diff']))
    xmax = max(abs(streaks['x_diff']))
    ymean = np.mean(abs(streaks['y_diff']))
    xmean = np.mean(abs(streaks['x_diff']))
    slope = ymax / xmax
    mean = np.mean(streaks['slope'])
    print(f'calculated slope is {slope}, mean slope from streaks is {mean}.')
    xrate = xmax / expt 
    yrate = ymax / expt # used to be ymax / xmax
    # Number of pixels to offset each image.
    x_offset, y_offset = int(xrate*.1)-5, int(yrate*.1)-5 
    frames = int((ones_frame)/.1)
    hdul = fits.open(f"/home/bekah/gphoton_working/test_data/e10982/e10982-nd-t00.1-b01-f{frames}-g_dose.fits.gz")
    img1 = hdul[0].data
    new_shape = ((layers - 1)*y_offset + img1.shape[0],
                 (layers - 1)*x_offset + img1.shape[1])  
    stacked = np.zeros(new_shape) #, dtype=np.float)
    stacked2 = np.zeros(new_shape) #, dtype=np.float)
    # adding image layers together 
    for layer in range(layers):
        print("adding image layers together")
        frame = frames + layer 
        frame2 = frames + layer + layers 
        dose_ais = fits.open(f"/home/bekah/gphoton_working/test_data/e10982/e10982-nd-t00.1-b01-f{frame}-g_dose.fits.gz")
        img1 = dose_ais[0].data #*layer # for tagging pixels as being from a layer  
        layer_op = (layers-1) - layer 
        stacked[layer_op*y_offset:layer_op*y_offset + img1.shape[0],
        layer_op*x_offset:layer_op*x_offset + img1.shape[1]] += img1
    # saving image to fits 
    hdu = fits.PrimaryHDU(stacked)
    hdul = fits.HDUList([hdu])
    hdul.writeto('stacked2.fits',overwrite=True)
    
    return 

In [ ]:
def getXYlist_stackedFrames():
    """Use DAOStarFinder to extract point sources from stacked frames and 
    produce a fits XYlist. """
    from photutils import DAOStarFinder
    # opening image for DAO starfinder 
    s = fits.open('/home/bekah/glcat/stacked2.fits')
    sim = s[0].data
    # trying dao star finder / xylist call to astrometry.net instead of 
    # using an image run 
    daofind = DAOStarFinder(fwhm=4, threshold=1, sharplo=0.00)
    star_list = daofind(sim)
    tbl = star_list.to_pandas()
    star_list = tbl.sort_values(by="flux", ascending=False)
    colx = fits.Column(name='X', format='E', array=tbl['xcentroid'])
    coly = fits.Column(name='Y', format='E', array=tbl['ycentroid'])
    hdu = fits.BinTableHDU.from_columns([colx, coly])
    tableName = "starlist_slew.fits"
    hdu.writeto(tableName, overwrite=True)
    print('getting image width and height')
    h, w = sim.shape
    print(f'height is {h} and width is {w}')
    return h, w


In [ ]:
# run astrometry.net 

def astrometry_xylist_slew(h, w): 
    
    import subprocess 

    cmd = f"solve-field --overwrite -w {w} -e {h} " \
          f"--scale-units arcsecperpix --scale-low 1.0 --scale-high 1.5 " \
          f"--radius 5 '/home/bekah/glcat/starlist_slew.fits'"

    subprocess.call(cmd, shell=True)
    
    print("Astrometry.net subprocess called.")
    
    return 

In [ ]:
# figure out any transformations required by center of image / pixel sitn 

def correct_aspect(): 
    """getting ra, dec solution for the center of the first frame 
    in the stacked series """
    
    return 
    

In [ ]:
def add_slew_to_aspect(eclipse, file_names):
    """adds slewframes to aspect table for eclipse so that the slew
    frames can be added to photonlists"""
    from pyarrow import parquet
    from gPhoton.io.mast import get_raw_paths, download_data 

    # loading aspect table to add time stamp and flags
    parq = parquet.read_table(file_names['old_aspect_parq'])
    aspect = parq.to_pandas()
    aspect = aspect[aspect["eclipse"] == eclipse]
    aspect = aspect.reset_index()

    #download scst 
    scst = download_data(eclipse, "scst", band, datadir=os.path.dirname(scstpath),)
    scst_pd = pd.DataFrame(scst[1].data)
    scst_pd = scst_pd.rename(columns={"pktime": "time"})

    #merge to get slew frames 
    merged_df = scst_pd.merge(aspect, how = 'left', on = ['time'])
    slew_frames = slew_frames[slew_frames['hvnom_nuv']==1]
    
    # use acs solns for ra / dec / roll for slew frames in asp soln 
    slew_frames['ra'] = slew_frames['ra'].fillna(slew_frames['ra_acs'])
    slew_frames['dec'] = slew_frames['dec'].fillna(slew_frames['dec_acs'])
    slew_frames['roll'] = slew_frames['roll'].fillna(slew_frames['roll_acs'])
    
    slew_frames = slew_frames.astype({'ra': 'float64',
                          'dec': 'float64',
                          'roll': 'float64'})

    # save to parquet
    slew_frames.to_parquet(file_names["aspect_parq"], compression=None)
    
    return

In [ ]:
# Get slew frames 

# loading aspect table 
parq  = parquet.read_table('/home/bekah/gphoton_working/gPhoton/aspect/aspect.parquet')
aspect = parq.to_pandas()
aspect = aspect[aspect["eclipse"]==eclipse]
aspect = aspect.reset_index()

# open scst file 
scst = fits.open("/home/bekah/gphoton_working/e10982-scst.fits.gz")
scst_pd = pd.DataFrame(scst[1].data)
scst_pd = scst_pd.rename(columns={"pktime": "time"})

merged_df = scst_pd.merge(aspect, how = 'left', on = ['time'])
slew_frames = merged_df[merged_df['ra'].isna()]
# voltage needs to be correct 
slew_frames = slew_frames[slew_frames['hvnom_nuv']==1]

slew_frames['ra'] = slew_frames['ra'].fillna(slew_frames['ra_acs'])
slew_frames['dec'] = slew_frames['dec'].fillna(slew_frames['dec_acs'])
slew_frames['roll'] = slew_frames['roll'].fillna(slew_frames['roll_acs'])

return slew_frames 